In [ ]:
import os
import numpy as np
os.environ['KAGGLE_USERNAME'] = "vedant"
os.environ['KAGGLE_KEY'] = "28739d1288d2cf227bb6aa942bee9820"

In [ ]:
!kaggle datasets download imrankhan77/autistic-children-facial-data-set

Dataset URL: https://www.kaggle.com/datasets/imrankhan77/autistic-children-facial-data-set
License(s): CC0-1.0
 89% 121M/137M [00:00<00:00, 162MB/s] 
100% 137M/137M [00:01<00:00, 135MB/s]


In [ ]:
!unzip /content/autistic-children-facial-data-set.zip

Streaming output truncated to the last 5000 lines.
  inflating: consolidated/autistic/0873.jpg  
  inflating: consolidated/autistic/0874.jpg  
  inflating: consolidated/autistic/0875.jpg  
  inflating: consolidated/autistic/0876.jpg  
  inflating: consolidated/autistic/0877.jpg  
  inflating: consolidated/autistic/0878.jpg  
  inflating: consolidated/autistic/0879.jpg  
  inflating: consolidated/autistic/0880.jpg  
  inflating: consolidated/autistic/0881.jpg  
  inflating: consolidated/autistic/0882.jpg  
  inflating: consolidated/autistic/0883.jpg  
  inflating: consolidated/autistic/0884.jpg  
  inflating: consolidated/autistic/0885.jpg  
  inflating: consolidated/autistic/0886.jpg  
  inflating: consolidated/autistic/0887.jpg  
  inflating: consolidated/autistic/0888.jpg  
  inflating: consolidated/autistic/0889.jpg  
  inflating: consolidated/autistic/0890.jpg  
  inflating: consolidated/autistic/0891.jpg  
  inflating: consolidated/autistic/0892.jpg  
  inflating: consolidated/aut

In [ ]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Function to detect faces in an image
def detect_faces(image_path, face_cascade):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    return [(x, y, w, h) for (x, y, w, h) in faces]

# Function to extract the face region from an image
def extract_face(image_path, face_cascade):
    faces = detect_faces(image_path, face_cascade)
    if len(faces) == 0:
        return None
    img = cv2.imread(image_path)
    x, y, w, h = faces[0]  # Assuming the first detected face
    face_img = img[y:y+h, x:x+w]
    return face_img

# Function to prepare the data: load, detect, and extract faces
def prepare_data(data_directory, face_cascade):
    faces = []
    labels = []
    label_names = os.listdir(data_directory)

    for label_name in label_names:
        label_dir = os.path.join(data_directory, label_name)
        for img_name in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_name)
            face_img = extract_face(img_path, face_cascade)
            if face_img is not None:
                faces.append(cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY))
                labels.append(label_name)

    return faces, labels, label_names

# Load OpenCV's pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Set the path to your dataset
data_directory = '/content/train'
faces, labels, label_names = prepare_data(data_directory, face_cascade)

# Encode labels to integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Train the LBPH face recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(encoded_labels))

# Function to recognize a face from a new image
def recognize_face(image_path, recognizer, face_cascade, label_encoder):
    face_img = extract_face(image_path, face_cascade)
    if face_img is not None:
        gray_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        label, confidence = recognizer.predict(gray_face)
        name = label_encoder.inverse_transform([label])[0]
        return name, confidence
    return None, None

# Example usage
test_image_path = '/content/Screenshot (5).png'
name, confidence = recognize_face(test_image_path, recognizer, face_cascade, label_encoder)
print(f'Predicted name: {name}, Confidence: {confidence}')


Predicted name: None, Confidence: None
